# Cross Validation

### Beatriz Loureiro 
### Giovanni Dalvi
### Richard Sousa Antunes

In [46]:
import sys
import importlib
sys.path.append('../')
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Lasso
import math
from sklearn.model_selection import cross_val_score, StratifiedKFold


from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.cross_validation.cross_validation import CrossValidation

In [47]:
# Defining time_index_col (must be the same column in all inputs) and keep_cols refering to the columns that will remain in the dataset
dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'HIGHT', 'LOW', 'CLOSE', 'VW','TRANSACTIONS'])

In [48]:
# Example Files
fname_AAPL = 'equities/AAPL_2020-04-07_2022-04-06.parquet'

# No. of Records from example dataset
N = 30000

In [49]:
# Dataset chosen in this simulation
ticker = 'AAPL'
fname = fname_AAPL

In [50]:
# loading assets into to an unique df
df = dataloader.load_dataset({ticker:'../data/'+fname}).iloc[:N]
display(df)

,AAPL_VOLUME,AAPL_OPEN,AAPL_HIGHT,AAPL_LOW,AAPL_CLOSE,AAPL_VW,AAPL_TRANSACTIONS
DATE,,,,,,,
2020-04-07 12:17:00,14256.0,67.3500,67.3625,67.3275,67.3375,67.3454,45
2020-04-07 12:18:00,16552.0,67.3501,67.3625,67.3250,67.3250,67.3407,57
2020-04-07 12:19:00,80172.0,67.3250,67.4475,67.3250,67.4225,67.3937,149
2020-04-07 12:20:00,183740.0,67.4475,67.4700,67.3125,67.3700,67.3671,159
2020-04-07 12:21:00,35372.0,67.3500,67.3850,67.2975,67.2975,67.3469,102
...,...,...,...,...,...,...,...
2020-06-05 15:42:00,282372.0,82.1317,82.1725,82.0925,82.1168,82.1329,769
2020-06-05 15:43:00,195168.0,82.1125,82.1250,82.0925,82.1100,82.1059,533
2020-06-05 15:44:00,290188.0,82.1075,82.2450,82.1050,82.2400,82.1737,756


In [51]:
X = df.drop('AAPL_VW', axis=1)
Y = df['AAPL_VW']

In [55]:
import warnings
warnings.filterwarnings('ignore')

CV = CrossValidation(overlap=0.01, embargo_rate = 0.01)

for k in range(3):
    alpha = (k+1)/10
    scores = CV.cross_validation_score(X, Y, Lasso(alpha=alpha), 'mean_squared_error')
    print('\nAlpha: ', alpha,'\nlista_scores: ', scores, '\nmédia score: ', np.mean(scores))
    scores_classico = -cross_val_score(Lasso(alpha=alpha), X, Y, cv=5, scoring='neg_mean_squared_error')
    print('\nCross-Validation normal do sklearn: ', scores_classico, '\nmédia score: ', np.mean(scores_classico))


Alpha:  0.1 
lista_scores:  [0.004634136827430485, 0.004089091422742909, 0.003016594164251107, 0.017680344146764417, 0.002930915236358706] 
média score:  0.0064702163595095254

Cross-Validation normal do sklearn:  [0.00448302 0.00406024 0.00301328 0.01762743 0.00284644] 
média score:  0.006406084767726789

Alpha:  0.2 
lista_scores:  [0.010161235985044568, 0.0064417237700909735, 0.0032451709356342466, 0.0191054993877452, 0.008286089622386545] 
média score:  0.009447943940180305

Cross-Validation normal do sklearn:  [0.00955868 0.00635083 0.00323903 0.01885634 0.00790306] 
média score:  0.00918158960043335

Alpha:  0.3 
lista_scores:  [0.019353209813130032, 0.010396305660153123, 0.003613536943075121, 0.021560707384795458, 0.017163499347664563] 
média score:  0.014417451829763658

Cross-Validation normal do sklearn:  [0.01799717 0.0102126  0.0036034  0.02096862 0.01631125] 
média score:  0.013818608972508362
